In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, GlobalAveragePooling2D

2024-07-29 05:54:14.523244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 05:54:14.523388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 05:54:14.670589: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import pandas as pd
labels = pd.read_csv('/kaggle/input/newcopy/new - Copy/dataset_information.csv') # paste it here
print(labels)

              ;Train;Test;Validation
0               Clean;4000;2000;2000
1          JavaScript;2363;1188;1214
2  JavaScript in HTML;2284;1167;1162
3          PowerShell;2468;1164;1213
4  Ethereum Addresses;2473;1247;1193
5    URL/IP addresses;2412;1234;1218
6              Total;16000;8000;8000


In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Hyperparameters
batch_size = 32
num_classes = 2  # cover and stego
input_shape = (512, 512, 3)

# Data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Validation and test data generator
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Define your train, validation, and test generators
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/newcopy/new - Copy/train/train',
    target_size=(512, 512),
    batch_size=batch_size,
    class_mode='binary'
)

val_generator = val_test_datagen.flow_from_directory(
    '/kaggle/input/newcopy/new - Copy/val/val',
    target_size=(512, 512),
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = val_test_datagen.flow_from_directory(
    '/kaggle/input/newcopy/new - Copy/test/test',
    target_size=(512, 512),
    batch_size=batch_size,
    class_mode='binary'
)

Found 24000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 12000 images belonging to 2 classes.


In [4]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    Conv2D(512, (3, 3), activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    Dropout(0.25),

    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # For binary classification
]) #training 85, testing 75


# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model.summary()
# Train the model
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)


# Evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}') #75
model.save('/kaggle/working/my_model.h5')


/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 512, 512, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 512, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 512, 512, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512, 512, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 256, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 256, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256, 256, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 128, 128, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128, 128, 256)  │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 64, 64, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 64, 64, 512)    │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 64, 64, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 4,956,225 (18.91 MB)

 Trainable params: 4,952,385 (18.89 MB)

...

Epoch 1/20


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-07-29 05:55:21.706312: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng35{k2=2,k3=0} for conv (f32[32,64,512,512]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,64,512,512]{3,2,1,0}, f32[64,64,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"conv_result_scale":1,"activation_mode":"kRelu","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-07-29 05:55:21.739019: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.032884585s
Trying algori

750/750 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.6262 - loss: 0.6726

W0000 00:00:1722234602.749871     126 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


750/750 ━━━━━━━━━━━━━━━━━━━━ 2254s 2s/step - accuracy: 0.6263 - loss: 0.6725 - val_accuracy: 0.7490 - val_loss: 0.7967
Epoch 2/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1678s 2s/step - accuracy: 0.8148 - loss: 0.3870 - val_accuracy: 0.7498 - val_loss: 5.3746
Epoch 3/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1680s 2s/step - accuracy: 0.8299 - loss: 0.3403 - val_accuracy: 0.7500 - val_loss: 1.9879
Epoch 4/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1704s 2s/step - accuracy: 0.8315 - loss: 0.3384 - val_accuracy: 0.7489 - val_loss: 0.4975
Epoch 5/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1677s 2s/step - accuracy: 0.8308 - loss: 0.3411 - val_accuracy: 0.6771 - val_loss: 0.6571
Epoch 6/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1677s 2s/step - accuracy: 0.8369 - loss: 0.3248 - val_accuracy: 0.7498 - val_loss: 25.7399
Epoch 7/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1674s 2s/step - accuracy: 0.8392 - loss: 0.3191 - val_accuracy: 0.7498 - val_loss: 11.7994
Epoch 8/20
750/750 ━━━━━━━━━━━━━━━━━━━━ 1672s 2s/step - accuracy: 0.8358 - loss: 0.3329 - val_accur